<h3>Scrap wikipedia to get the tickers</h3>

In [1]:
import time
from bs4 import BeautifulSoup
import requests_html
import pandas as pd
import psycopg2
import yfinance as yf

**IF THE FILE NAMED 'tickers.csv' DOESN'T EXISTS PROCEED WITH THIS CELL, ELSE GO TO THE NEXT**

In [ ]:
wikipedia_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

df_sp_500 = pd.read_html(wikipedia_url)[0]

df_sp_500 = df_sp_500.rename(columns={'Security':'Name'})
df_sp_500 = df_sp_500.drop(columns=['SEC filings', 
                      'CIK', 
                      'Date first added', 
                      'Founded', 
                      'Headquarters Location', 
                      'GICS Sector', 
                      'GICS Sub-Industry' ])

df_nasdaq = pd.read_csv('nasdaq.csv')
df_nasdaq = df_nasdaq.drop(columns=['Last Sale', 
                                    'Net Change', 
                                    '% Change', 
                                    'Market Cap', 
                                    'Country', 
                                    'IPO Year', 
                                    'Volume', 
                                    'Sector', 
                                    'Industry'])

df_tickers = pd.concat([df_sp_500, df_nasdaq])
df_tickers.sort_values("Symbol", inplace=True)
df_tickers.drop_duplicates(keep=False,inplace=True)
df_tickers.to_csv('tickers.csv', index=False)

Having the file with the tickers read it into a df and get the info from YF

In [2]:
all_tickers_df = pd.read_csv('tickers.csv')
all_tickers = pd.Series(all_tickers_df['Symbol'])
all_tickers.unique()
all_tickers = set(all_tickers)

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=tickers user=ticker password=ticker")
conn.set_session(autocommit=True)

cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS symbols (Symbol text PRIMARY KEY, Name text, Sector text, Industry text, Currency text, Volume int, TimeZone text, Market text);")
cur.execute("CREATE TABLE IF NOT EXISTS symbols_processed (Symbol text PRIMARY KEY);")

symbols_processed = set()
cur.execute("SELECT Symbol FROM symbols_processed")
row = cur.fetchone()
while row:
    symbols_processed.add(row[0])
    row = cur.fetchone()

In [4]:
remaining_symbols = all_tickers.difference(symbols_processed)
print("All tickers: " + str(len(all_tickers)))
print("Processed tickers: " + str(len(symbols_processed)))
print("Remaining tickers: " + str(len(remaining_symbols)))

All tickers: 7652
Processed tickers: 0
Remaining tickers: 7652


In [5]:
def insert_into_db(data) :
    try :
        cur.execute("INSERT INTO symbols (Symbol, Name, Sector, Industry, Currency, Volume, TimeZone, Market) \
                     VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", \
                     (data['symbol'], data['name'], data['sector'], data['industry'], data['currency'], data['volume'], data['exchangeTimezoneShortName'], data['market']))
    except psycopg2.Error as e:
        print(f'NOT INSERTED: {data}')
        print("Error inserting values")
        print(e)

In [6]:
def insert_into_processed(symbol) :
    try :
        cur.execute("INSERT INTO symbols_processed (Symbol) \
                     VALUES (%s)", \
                     (symbol,))
    except psycopg2.Error as e:
        print("Error inserting symbol")
        print(e)

In [7]:
def populate_info(symbol) :
    try :
        tick = yf.Ticker(symbol).info
        tick_info = {}
        tick_info['symbol'] = symbol

        if 'shortName' in tick :
            tick_info['name'] = tick['shortName']
        else :
            print(f'Populate info RETURN: {symbol}')
            return
        if 'sector' in tick :
            tick_info['sector'] = tick['sector']
        else :
            tick_info['sector'] = None
        if 'industry' in tick :
            tick_info['industry'] = tick['industry']
        else :
            tick_info['industry'] = None
        if 'volume' in tick :
            tick_info['volume'] = tick['volume']
        else : 
             tick_info['volume'] = None
        if 'shortName' in tick :
            tick_info['shortName'] = tick['shortName']
        else :
            tick_info['shortName'] = None
        if 'exchangeTimezoneShortName' in tick :
            tick_info['exchangeTimezoneShortName'] = tick['exchangeTimezoneShortName']
        else :
            tick_info['exchangeTimezoneShortName'] = None
        if 'market' in tick :
            tick_info['market'] = tick['market']
        else :
            tick_info['market'] = None
        if 'currency' in tick :
            tick_info['currency'] = tick['currency']
        else :
            tick_info['currency'] = None

        insert_into_db(tick_info)
    except ValueError as e :
        print("error processing {}".format(symbol))
    finally :
        insert_into_processed(symbol)
        symbols_processed.add(symbol)

In [ ]:
for ticker in remaining_symbols :
    symbol = ticker.replace(".","-")
    symbol = symbol.split("^")[0]
    if symbol not in symbols_processed :
        populate_info(symbol)

In [ ]:
cur.execute("SELECT * FROM symbols")
row = cur.fetchone()
sectors = []
while row:
    sectors.append(row)
    print(row)
    row = cur.fetchone()

In [ ]:
cur.execute("SELECT Sector, count(1) as c FROM symbols GROUP BY Sector ORDER BY c DESC")
row = cur.fetchone()
sectors = []
while row:
    sectors.append(row[0])
    print(row)
    row = cur.fetchone()

In [ ]:
cur.execute("DROP TABLE symbols")
cur.execute("DROP TABLE symbols_processed")
cur.close()
conn.close()